In [1]:
# Celda 1: Importar bibliotecas y cargar datos
import pandas as pd
import numpy as np
import math
import random
import csv

# Cargar el conjunto de datos
try:
    df = pd.read_csv('parkinsons_data.csv')
    print("Archivo 'parkinsons_data.csv' cargado exitosamente.")
except FileNotFoundError:
    print("Error: Asegúrate de haber subido el archivo 'parkinsons_data.csv' a Google Colab.")

# Celda 2: Inspección inicial del DataFrame
print("--- Información General del DataFrame ---")
df.info()

print("\n\n--- Primeras 5 filas del DataFrame ---")
print(df.head())

# Celda 3: Tarea 1.1 - Verificar valores perdidos
print("\n--- Verificación de Valores Perdidos ---")
valores_perdidos = df.isnull().sum()
if valores_perdidos.sum() == 0:
    print("No se encontraron valores perdidos en el DataSet.")
else:
    print("Se encontraron los siguientes valores perdidos por columna:")
    print(valores_perdidos[valores_perdidos > 0])

# Celda 4: Tarea 1.2 - Verificar desbalance de clases
print("\n--- Verificación de Desbalance de Clases ---")
# Asumimos que la columna 'status' es la clase. Si es otra, se debe cambiar aquí.
columna_clase = 'status'
conteo_clases = df[columna_clase].value_counts()
print(f"Distribución de clases en la columna '{columna_clase}':")
print(conteo_clases)

total_muestras = len(df)
porcentaje_clase_0 = (conteo_clases[0] / total_muestras) * 100
porcentaje_clase_1 = (conteo_clases[1] / total_muestras) * 100

print(f"\nPorcentaje Clase 0: {porcentaje_clase_0:.2f}%")
print(f"Porcentaje Clase 1: {porcentaje_clase_1:.2f}%")

if abs(porcentaje_clase_0 - porcentaje_clase_1) > 20: # Umbral del 20% de diferencia
    print("\nConclusión: El DataSet está desbalanceado.")
else:
    print("\nConclusión: El DataSet parece estar razonablemente balanceado.")

Archivo 'parkinsons_data.csv' cargado exitosamente.
--- Información General del DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    int64  
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 1

In [2]:
# Celda 5: Preparación de Datos
# Eliminar la columna 'name' ya que no es una característica numérica para el modelo
if 'name' in df.columns:
    df = df.drop(columns=['name'])

# Separar características (X) y la etiqueta (y)
X = df.drop(columns=[columna_clase]).values
y = df[columna_clase].values

# Celda 6: Implementación de Funciones y Clases desde Cero

# --- Normalizador de Datos ---
# Es crucial para que los algoritmos de redes neuronales converjan adecuadamente.
def min_max_scaler(data):
    min_vals = data.min(axis=0)
    max_vals = data.max(axis=0)
    # Evitar división por cero si una columna tiene el mismo valor en todas las filas
    range_vals = np.where(max_vals - min_vals == 0, 1, max_vals - min_vals)
    scaled_data = (data - min_vals) / range_vals
    return scaled_data, min_vals, range_vals

def scale_test_data(data, min_vals, range_vals):
    return (data - min_vals) / range_vals

# --- Función de Segmentación: Hold-Out Estratificado ---
def stratified_hold_out_split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        random.seed(random_state)

    X_train, X_test, y_train, y_test = [], [], [], []
    indices_por_clase = {clase: [i for i, label in enumerate(y) if label == clase] for clase in np.unique(y)}

    for clase, indices in indices_por_clase.items():
        random.shuffle(indices)
        n_test = int(len(indices) * test_size)
        indices_test = indices[:n_test]
        indices_train = indices[n_test:]

        X_test.extend(X[indices_test])
        y_test.extend(y[indices_test])
        X_train.extend(X[indices_train])
        y_train.extend(y[indices_train])

    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

# --- Implementación del Perceptrón Multicapa (MLP) ---
class MLP_Desde_Cero:
    def __init__(self, tam_capas, tasa_aprendizaje=0.01, epocas=300):
        # tam_capas es una lista, ej: [n_entradas, n_oculta1, n_salida]
        self.tam_capas = tam_capas
        self.tasa_aprendizaje = tasa_aprendizaje
        self.epocas = epocas
        self.pesos = []
        self.sesgos = []

        # Inicialización de pesos y sesgos
        for i in range(len(tam_capas) - 1):
            # Se inicializan los pesos con valores aleatorios pequeños para romper la simetría
            w = np.random.randn(tam_capas[i], tam_capas[i+1]) * 0.1
            b = np.zeros((1, tam_capas[i+1]))
            self.pesos.append(w)
            self.sesgos.append(b)

    def _sigmoid(self, x):
        # Función de activación sigmoide, útil para capas de salida en clasificación binaria
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def _sigmoid_derivada(self, x):
        # Derivada de la sigmoide, necesaria para la retropropagación
        return x * (1 - x)

    def _forward_pass(self, X):
        # Propagación hacia adelante
        activaciones = [X]
        entrada_capa = X
        for i in range(len(self.pesos)):
            salida_lineal = np.dot(entrada_capa, self.pesos[i]) + self.sesgos[i]
            activacion = self._sigmoid(salida_lineal)
            activaciones.append(activacion)
            entrada_capa = activacion
        return activaciones

    def _backward_pass(self, y, activaciones):
        # Retropropagación del error para calcular gradientes
        errores = [y.reshape(-1, 1) - activaciones[-1]]
        deltas = [errores[-1] * self._sigmoid_derivada(activaciones[-1])]

        # Iterar hacia atrás desde la penúltima capa
        for i in range(len(activaciones) - 2, 0, -1):
            error = np.dot(deltas[0], self.pesos[i].T)
            delta = error * self._sigmoid_derivada(activaciones[i])
            deltas.insert(0, delta)

        return deltas

    def _actualizar_pesos(self, activaciones, deltas):
        # Actualizar pesos y sesgos usando los gradientes calculados
        for i in range(len(self.pesos)):
            self.pesos[i] += np.dot(activaciones[i].T, deltas[i]) * self.tasa_aprendizaje
            self.sesgos[i] += np.sum(deltas[i], axis=0, keepdims=True) * self.tasa_aprendizaje

    def fit(self, X, y):
        for _ in range(self.epocas):
            activaciones = self._forward_pass(X)
            deltas = self._backward_pass(y, activaciones)
            self._actualizar_pesos(activaciones, deltas)

    def predict(self, X):
        activaciones = self._forward_pass(X)
        # La predicción es la salida de la última capa
        predicciones_prob = activaciones[-1]
        # Convertir probabilidades a clases binarias (0 o 1)
        predicciones_clase = [1 if p > 0.5 else 0 for p in predicciones_prob]
        return np.array(predicciones_clase)

# --- Implementación de K-Nearest Neighbors (KNN) ---
class KNN_Desde_Cero:
    def __init__(self, k=3):
        self.k = k
        self.X_train = None
        self.y_train = None

    def _distancia_euclidiana(self, p1, p2):
        # Cálculo de la distancia entre dos puntos
        return np.sqrt(np.sum((p1 - p2)**2))

    def fit(self, X_train, y_train):
        # En KNN, "fit" solo significa memorizar los datos de entrenamiento
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predicciones = []
        for punto_test in X_test:
            # Calcular distancias a todos los puntos de entrenamiento
            distancias = [self._distancia_euclidiana(punto_test, punto_train) for punto_train in self.X_train]
            # Obtener los índices de los k vecinos más cercanos
            indices_vecinos = np.argsort(distancias)[:self.k]
            # Obtener las etiquetas de esos vecinos
            etiquetas_vecinos = [self.y_train[i] for i in indices_vecinos]
            # Predecir por voto de mayoría
            prediccion = max(set(etiquetas_vecinos), key=etiquetas_vecinos.count)
            predicciones.append(prediccion)
        return np.array(predicciones)

# --- Métricas de Desempeño ---
def calcular_metricas(y_true, y_pred):
    # Calcula TP, TN, FP, FN
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    # Calcula métricas, con manejo de división por cero
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0 # También llamado Sensibilidad
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "Specificity": specificity
    }

In [3]:
# Celda 7: Lógica para Leave-One-Out Cross-Validation (LOOCV)
# LOOCV es un caso especial de K-Fold donde k = N (número de muestras)
print("\n--- Iniciando Ejecución con Leave-One-Out (LOOCV) ---")
print("ADVERTENCIA: Este proceso es muy lento y puede tardar varias horas, especialmente para los modelos MLP.")

# Parámetros
N_MUESTRAS = len(X)
N_CORRIDAS_MLP = 30 # Para MLP por su inicialización aleatoria
N_ENTRADAS = X.shape[1]
N_SALIDAS = 1

resultados_loocv = {
    "MLP (10 neuronas)": [], "MLP (100 neuronas)": [],
    "MLPDeep (10-10-10)": [], "MLPDeep (100-100-100)": [],
    "KNN (k=3)": [], "KNN (k=5)": [], "KNN (k=27)": []
}

# --- Ejecución para KNN (solo 1 vez, es determinista) ---
print("\nIniciando LOOCV para modelos KNN...")
for k_val in [3, 5, 27]:
    y_preds_knn = []
    for i in range(N_MUESTRAS):
        # Separar datos
        X_test = X[i, :].reshape(1, -1)
        y_test = y[i]
        X_train = np.delete(X, i, axis=0)
        y_train = np.delete(y, i, axis=0)

        knn = KNN_Desde_Cero(k=k_val)
        knn.fit(X_train, y_train)
        pred = knn.predict(X_test)
        y_preds_knn.append(pred[0])

    # Calcular métricas una vez se tienen todas las predicciones
    metricas = calcular_metricas(y, np.array(y_preds_knn))
    resultados_loocv[f"KNN (k={k_val})"].append(metricas)
print("LOOCV para KNN completado.")

# --- Ejecución para MLP (30 corridas) ---
modelos_mlp_config = {
    "MLP (10 neuronas)": {"capas": [N_ENTRADAS, 10, N_SALIDAS]},
    "MLP (100 neuronas)": {"capas": [N_ENTRADAS, 100, N_SALIDAS]},
    "MLPDeep (10-10-10)": {"capas": [N_ENTRADAS, 10, 10, 10, N_SALIDAS]},
    "MLPDeep (100-100-100)": {"capas": [N_ENTRADAS, 100, 100, 100, N_SALIDAS]}
}

for nombre_modelo, config in modelos_mlp_config.items():
    print(f"\nIniciando {N_CORRIDAS_MLP} corridas LOOCV para {nombre_modelo}...")
    for corrida in range(N_CORRIDAS_MLP):
        y_preds_mlp = []
        print(f"  Corrida {corrida+1}/{N_CORRIDAS_MLP} para {nombre_modelo}...")
        for i in range(N_MUESTRAS):
            # Separar datos
            X_test_raw = X[i, :].reshape(1, -1)
            y_test = y[i]
            X_train_raw = np.delete(X, i, axis=0)
            y_train = np.delete(y, i, axis=0)

            # Normalizar
            X_train_scaled, min_v, range_v = min_max_scaler(X_train_raw)
            X_test_scaled = scale_test_data(X_test_raw, min_v, range_v)

            # Entrenar y predecir
            mlp = MLP_Desde_Cero(tam_capas=config["capas"], epocas=50) # Menos épocas por la carga
            mlp.fit(X_train_scaled, y_train)
            pred = mlp.predict(X_test_scaled)
            y_preds_mlp.append(pred[0])

        # Calcular y guardar métricas para la corrida
        metricas = calcular_metricas(y, np.array(y_preds_mlp))
        resultados_loocv[nombre_modelo].append(metricas)
    print(f"LOOCV para {nombre_modelo} completado.")

# Celda 8: Cálculo de Estadísticas y Exportación para LOOCV
print("\n--- Calculando Estadísticas (Promedio, Mínimo, Máximo) para LOOCV ---")
estadisticas_finales_loocv = []

for modelo, metricas_corridas in resultados_loocv.items():
    # Para KNN, solo hay una corrida
    if "KNN" in modelo and len(metricas_corridas) == 1:
        metricas_corridas = metricas_corridas * N_CORRIDAS_MLP # Replicamos para consistencia estadística

    df_metricas = pd.DataFrame(metricas_corridas)

    promedio = df_metricas.mean().to_dict()
    minimo = df_metricas.min().to_dict()
    maximo = df_metricas.max().to_dict()

    for metrica in ["Accuracy", "Precision", "Recall", "Specificity"]:
        estadisticas_finales_loocv.append({
            "Validation": "Leave-One-Out",
            "Algorithm": modelo,
            "Metric": metrica,
            "Mean": promedio[metrica],
            "Min": minimo[metrica],
            "Max": maximo[metrica]
        })

df_estadisticas_loocv = pd.DataFrame(estadisticas_finales_loocv)
print(df_estadisticas_loocv)

# Exportar a CSV
df_estadisticas_loocv.to_csv('loocv_results.csv', index=False)
print("\nResultados guardados en 'loocv_results.csv'")


--- Iniciando Ejecución con Leave-One-Out (LOOCV) ---
ADVERTENCIA: Este proceso es muy lento y puede tardar varias horas, especialmente para los modelos MLP.

Iniciando LOOCV para modelos KNN...
LOOCV para KNN completado.

Iniciando 30 corridas LOOCV para MLP (10 neuronas)...
  Corrida 1/30 para MLP (10 neuronas)...
  Corrida 2/30 para MLP (10 neuronas)...
  Corrida 3/30 para MLP (10 neuronas)...
  Corrida 4/30 para MLP (10 neuronas)...
  Corrida 5/30 para MLP (10 neuronas)...
  Corrida 6/30 para MLP (10 neuronas)...
  Corrida 7/30 para MLP (10 neuronas)...
  Corrida 8/30 para MLP (10 neuronas)...
  Corrida 9/30 para MLP (10 neuronas)...
  Corrida 10/30 para MLP (10 neuronas)...
  Corrida 11/30 para MLP (10 neuronas)...
  Corrida 12/30 para MLP (10 neuronas)...
  Corrida 13/30 para MLP (10 neuronas)...
  Corrida 14/30 para MLP (10 neuronas)...
  Corrida 15/30 para MLP (10 neuronas)...
  Corrida 16/30 para MLP (10 neuronas)...
  Corrida 17/30 para MLP (10 neuronas)...
  Corrida 18/30 p